<a href="https://colab.research.google.com/github/taniasuchi07/Technical-Assessment/blob/main/AI_Engineer_(Level_1)_%E2%80%94_Technical_Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Setup**

In [6]:
!pip install -U langchain langchain-community


In [7]:
pip install torch torchvision torchaudio


In [8]:
pip install langchain faiss-cpu sentence-transformers pypdf nltk openai transformers


# **Load and Clean PDF Data**

In [10]:
from google.colab import files
uploaded = files.upload()

Saving HSC26_Bangla_1st_Paper.pdf to HSC26_Bangla_1st_Paper.pdf


In [11]:
import os
print(os.listdir())  # Show PDF

['.config', 'HSC26_Bangla_1st_Paper.pdf', 'sample_data']


In [14]:
print(len(pages))  # Should be > 0
print(pages[0].page_content[:500])  # See a preview

366



In [15]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re

# Load PDF
loader = PyPDFLoader("HSC26_Bangla_1st_Paper.pdf")
pages = loader.load()

# Basic cleaning
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # Normalize whitespace
    text = re.sub(r'[^\u0980-\u09FF\u0000-\u007F]+', '', text)  # Keep Bangla + English chars
    return text.strip()

cleaned_docs = [clean_text(doc.page_content) for doc in pages]



In [16]:
print(cleaned_docs[:3])  # Should show usable Bangla/English text

['', '', '']


In [18]:
pip install pytesseract pdf2image


In [24]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
# Install Poppler
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.8 [186 kB]
Fetched 186 kB in 1s (316 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126284 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.8_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.8) ...
Setting up poppler-utils (22.02.0-2ubuntu0.8) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
from pdf2image import convert_from_path
import pytesseract


# Install Poppler
!apt-get install -y poppler-utils

from pdf2image import convert_from_path

images = convert_from_path("/content/drive/MyDrive/Colab Notebooks/HSC26_Bangla_1st_Paper.pdf")

print(f"Converted {len(images)} pages")



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.8).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


# **Chunking & Embedding**

In [4]:
!apt-get install -y tesseract-ocr tesseract-ocr-ben


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
The following NEW packages will be installed:
  tesseract-ocr-ben
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 516 kB of archives.
After this operation, 870 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-ben all 1:4.00~git30-7274cfa-1.1 [516 kB]
Fetched 516 kB in 1s (689 kB/s)
Selecting previously unselected package tesseract-ocr-ben.
(Reading database ... 126314 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-ben_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-ocr-ben (1:4.00~git30-7274cfa-1.1) ...
Setting up tesseract-ocr-ben (1:4.00~git30-7274cfa-1.1) ...


In [ ]:
!apt-get install -y poppler-utils tesseract-ocr tesseract-ocr-ben
!pip install pdf2image pytesseract langchain langchain-community faiss-cpu sentence-transformers

# 📂 STEP 2: Import Libraries
from pdf2image import convert_from_path
import pytesseract
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
import re

# 🧹 STEP 3: Utility Function - Clean Text
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # remove extra spaces
    text = re.sub(r'[^\u0980-\u09FF\u0000-\u007F]+', '', text)  # keep Bangla + English
    return text.strip()

# 📄 STEP 4: Convert PDF to Images
pdf_path = "/content/drive/MyDrive/Colab Notebooks/HSC26_Bangla_1st_Paper.pdf"  # Upload the file first
images = convert_from_path(pdf_path)

# 🔍 STEP 5: Bangla OCR
ocr_text = []
for img in images:
    text = pytesseract.image_to_string(img, lang='ben')  # 'ben' = Bangla
    ocr_text.append(text)

# ✂️ STEP 6: Clean & Chunk
cleaned_docs = [clean_text(t) for t in ocr_text if len(t.strip()) > 0]
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = splitter.create_documents(cleaned_docs)

# 💡 STEP 7: Embedding Model (multilingual)
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# 🔎 STEP 8: Create Vector Store
vectorstore = FAISS.from_documents(documents, embedding_model)
vectorstore.save_local("bangla_knowledge_base")

print("✅ PDF successfully processed and stored in vector DB.")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
tesseract-ocr-ben is already the newest version (1:4.00~git30-7274cfa-1.1).
poppler-utils is already the newest version (22.02.0-2ubuntu0.8).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI  # Use your OpenAI key

# Load vectorstore
vectorstore = FAISS.load_local("bangla_knowledge_base", embedding_model)

# LLM setup (use your key)
import os
os.environ["OPENAI_API_KEY"] = "sk-..."  # Replace with your key
llm = OpenAI(temperature=0.2)

qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore.as_retriever())

# Example Bangla Query
question = "অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?"
result = qa_chain.run(question)
print(f"Q: {question}\nA: {result}")
